In [1]:
import tweepy
import pandas as pd
import os
import unidecode
#import datetime
from datetime import timedelta, datetime 
from dateutil.relativedelta import relativedelta
import time
import datetime as dt 
import pymongo
from pymongo import MongoClient
import numpy as np
from nltk.corpus import wordnet as wn
from nltk import sent_tokenize
from googletrans import Translator
pd.options.mode.chained_assignment = None 

In [2]:
client = tweepy.Client(bearer_token = "AAAAAAAAAAAAAAAAAAAAAKKEcwEAAAAA1VgzT5upnu%2B5POQ3dbdUx0%2B5vfA%3DdslAW6ut31gZhQD9SjNKJ4scCFd8bHgbevp8eGcJprbd2LTrGf", wait_on_rate_limit = True)

In [3]:
sel = pd.read_csv('../Data/SEL/SEL_full.txt', delimiter='\t', encoding= 'ISO-8859-1')

In [69]:
emociones = list(sel['Categoría'].unique())
emociones_dict ={}
for emocion in emociones:
    emociones_dict[emocion] = list(sel[sel['Categoría'] == emocion].sort_values(' PFA',  ascending=False)['Palabra'])
    for palabra in emociones_dict[emocion]:
        if not palabra.isascii():
            emociones_dict[emocion].append(unidecode.unidecode(palabra))

In [40]:
emotion_words = '(' +' OR '.join(emociones_dict['Sorpresa']) + ')'

In [70]:
emociones_dict

{'Alegría': ['felicidad',
  'divertidísimamente',
  'alegría',
  'divertido',
  'alegremente',
  'alegre',
  'contento',
  'amor',
  'amistoso',
  'amistad',
  'amartelado',
  'alegrar',
  'felizmente',
  'feliz',
  'afectuosamente',
  'sonriente',
  'divertidamente',
  'divertir',
  'celebrar',
  'afectuosidad',
  'risa',
  'bienestar',
  'amorosamente',
  'carcajada',
  'fascinante',
  'afortunado',
  'risueño',
  'alborozar',
  'alborozo',
  'agradable',
  'amar',
  'afecto',
  'bizbirondo',
  'amistosamente',
  'beneficio',
  'afectuoso',
  'bueno',
  'amigo',
  'amoroso',
  'fascinado',
  'amorosidad',
  'fascinación',
  'amigable',
  'adorable',
  'amigabilidad',
  'disfrute',
  'animado',
  'adorablemente',
  'satisfecho',
  'genial',
  'entusiasta',
  'estupendo',
  'beneficioso',
  'satisfacción',
  'placer',
  'excitante',
  'éxito',
  'enamoramiento',
  'entusiasmado',
  'amigablemente',
  'afectivo',
  'triunfo',
  'complacido',
  'entusiasmo',
  'cariño',
  'armonía',
  'á

In [24]:
trans_words =[]
translator = Translator()
for i in emociones_dict['Alegría']:
    data = i
    token = sent_tokenize(data)
    translatedText = translator.translate(token[0], dest="en", src= 'es')
    trans_words.append(translatedText.text)

In [30]:
word = trans_words[0]
forms = set() #We'll store the derivational forms in a set to eliminate duplicates
for happy_lemma in wn.lemmas('feliz'): #for each "happy" lemma in WordNet
    forms.add(happy_lemma.name()) #add the lemma itself
    for related_lemma in happy_lemma.derivationally_related_forms(): #for each related lemma
        forms.add(related_lemma.name()) #add the related lemma

In [34]:
variations_dict = {}
for i in trans_words:
    forms = set() #We'll store the derivational forms in a set to eliminate duplicates
    for happy_lemma in wn.lemmas(i): #for each "happy" lemma in WordNet
        forms.add(happy_lemma.name()) #add the lemma itself
        for related_lemma in happy_lemma.derivationally_related_forms(): #for each related lemma
            forms.add(related_lemma.name()) #add the related lemma
    variations_dict[i] = forms

happiness
hilarious
happiness
funny
happily
cheerful
glad
love
friendly
friendship
hilarious
joy


In [ ]:
trans_words =[]
translator = Translator()
for i in emociones_dict['Alegría']:
    data = i
    token = sent_tokenize(data)
    translatedText = translator.translate(token[0], dest="en", src= 'es')
    trans_words.append(translatedText.text)

In [43]:
trans_variations = {}
for i in variations_dict:
    trans_words_variations =[]
    for j in variations_dict[i]:
        token = sent_tokenize(j)
        translatedText = translator.translate(token[0], dest="es", src= 'en')
        trans_words_variations.append(translatedText.text)
    trans_variations[i] = trans_words_variations

In [47]:
lista = []
lista += emociones_dict['Alegría']
for i in trans_variations:
    lista += trans_variations[i]

In [51]:
new_emotion_list = list(set(lista))
new_emotion_words = '(#' +' OR #'.join(new_emotion_list) + ')'

In [52]:
new_emotion_words

'(#felizmente OR #amable OR #jubiloso OR #gracioso OR #divertidísimamente OR #amigable OR #alegria OR #amante OR #divertidísimo OR #chiste OR #felicidad OR #divertido OR #alegremente OR #alegría OR #contento OR #amistad OR #amabilidad OR #amistoso OR #adorable OR #alegre OR #hilaridad OR #divertida OR #amor OR #amigo OR #divertidisimamente)'

In [41]:
def get_trendings(trending, start_date, number_days, minutes_spam ):
    trending_words = trending + ' place_country:CO ' +  '-is:retweet'
    df_list = []
    end_date = start_date + relativedelta(days=number_days)
    start_time = dt.datetime.now()
    while start_date < end_date:
        now = dt.datetime.now()
        print('Extraction Date:', start_date, 'Trending:',trending, 'Start Time',start_time,  'Time extraction:',now, end="\r")
        upper_limit = start_date+ relativedelta(minutes=minutes_spam)
        start= start_date.isoformat("T") + "Z"
        end = upper_limit.isoformat("T") + "Z"
        rsp = client.search_all_tweets(query = trending_words ,start_time = start, end_time = end, sort_order= 'relevancy',
                                     tweet_fields = fields, max_results=500)
        time.sleep(3)
        tweets_df = pd.DataFrame(rsp.data, columns = fields)
        tweets_df['date']= start
        tweets_df = pd.concat([tweets_df.drop(['public_metrics'], axis=1), pd.json_normalize(tweets_df['public_metrics'])], axis=1)
        tweets_df = pd.concat([tweets_df.drop(['geo'], axis=1), pd.json_normalize(tweets_df['geo'])], axis=1)
        df_list.append(tweets_df)
        start_date = upper_limit
    trendings_df = pd.concat(df_list)
    trendings_df = trendings_df.reset_index(drop=True)
    return(trendings_df)

In [23]:
emotion_words

'(enrabiar OR enojo OR enojar OR enojado OR ira OR furiosamente OR violencia OR violento OR enrabietar OR odiar OR odio OR enfurecer OR enojosamente OR enojoso OR enfadado OR coraje OR ataque OR hartar OR rabia OR desquiciar OR enojamiento OR irritable OR enfadar OR enfado OR repudiar OR furia OR agresivo OR enojadizo OR enfierecido OR enfurecimiento)'

In [42]:
trending = emotion_words
fields = ['lang',
 'public_metrics',
 'author_id',
 'id',
 'created_at',
 'text',
         'conversation_id',
         'geo',
         'in_reply_to_user_id']
start_date = '2022-05-22'
number_days = 1
minutes_spam = 1440
start_date_format = datetime.strptime(start_date, '%Y-%m-%d')
trendings_df  = get_trendings(trending, start_date_format, number_days, minutes_spam )

In [46]:
df = pd.read_parquet('../Data/Trendings/2022-05-22/2022-05-22__#FedericoEsColombia.gzip')

In [71]:
mylist = emociones_dict['Tristeza']
pattern = '|'.join(mylist)

In [58]:
pattern

'sorpresivamente|sorprendente|asombrosamente|asombroso|sorpresa|sorpresivo|maravilloso|sorprender|asombro|asombrar|increible|mirífico|maravillosamente|asombramiento|sorprendentemente|estupefacto|increíblemente|impresionante|maravilla|inaudito|maravillar|impresionar|despampanante|espantar|espanto|atónito|boquiabierto|descomunalmente|estupefacción|anonadamiento|mirifico|increiblemente|atonito|estupefaccion'

In [72]:
list(df[df['text'].str.contains(pattern)]['text'])

['#FedericoEsColombia\nVamos a ganar la presidencia!!! Tiene más reversa un paracaídas https://t.co/rZmnLNvpVS',
 '#FedericoEsColombia y oportunidad, libertad y ganas de trabajar por los más humildes. Soy Paisa y fui testigo de su trabajo en la alcaldía. https://t.co/HoMtOCOEPS',
 '#FedericoEsColombia es la unión, es desenredar la vida de odios pensando en el progreso del país, es pensar en el campo con campesinos sembrando comida y no drogas, es pensar en el estudio de nuestros hijos y el cambio con oportunidades de trabajo.\nDios bendiga a #FicoPresidente https://t.co/KR9wZiLqdC',
 '@jarizabaletaf Petro quiere crear cortinas de humo con atentados y golpes de Estado falsos, pues sabe que va a perder.\nLas gente está mostrando su predilección y ya se decidió por @FicoGutierrez, con asistencia masivas a todas las convocatorias en las plazas.\n#FedericoEsColombia',
 '#FedericoEsColombia, si! Sobre sus hombros la única posibilidad de salvar a 🇨🇴 de caer en las garras del comunismo. Urnas 

In [62]:
from pyinflect import getAllInflections, getInflection
getAllInflections('love')

{'NN': ('love',),
 'NNS': ('loves',),
 'VB': ('love',),
 'VBP': ('love',),
 'VBD': ('loved',),
 'VBN': ('loved',),
 'VBG': ('loving',),
 'VBZ': ('loves',)}

In [54]:
forms = set() #We'll store the derivational forms in a set to eliminate duplicates
for happy_lemma in wn.lemmas("happy"): #for each "happy" lemma in WordNet
    forms.add(happy_lemma.name()) #add the lemma itself
    for related_lemma in happy_lemma.derivationally_related_forms(): #for each related lemma
        forms.add(related_lemma.name()) #add the related lemma

integrity


In [13]:
token

['entereza']

In [44]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/juanjo/nltk_data...


True

In [53]:
wn.lemmas("happy")[0].derivationally_related_forms()

[Lemma('happiness.n.01.happiness'), Lemma('happiness.n.02.happiness')]

In [5]:
def get_tweets(emotion, words_list, number_tweets, start, end):
    emotion_words = '(' +' OR '.join(words_list) + ')'
    searched_tweets = [tweet for tweet in tweepy.Paginator(client.search_all_tweets, query = emotion_words + ' place_country:CO' ,start_time = start, end_time = end, 
                                max_results=500).flatten(limit=number_tweets)]
    tweets_df = pd.DataFrame(searched_tweets, columns = ['text'])
    tweets_df['emotion']= emotion
    tweets_df['date']= start
    return tweets_df

In [9]:
month = '04'
start_date = datetime.strptime('2021-' + month, '%Y-%m')
end_date = start_date + relativedelta(months=1)
df_list = []
while start_date < end_date:
    print(start_date)
    next_date = start_date + relativedelta(days=1)
    start_date_standard = start_date.isoformat("T") + "Z"
    next_date_standard = next_date.isoformat("T") + "Z"
    for i in emociones_dict.keys():
        print(i)
        df_list.append(get_tweets(i, emociones_dict[i], 100, start_date_standard, next_date_standard ))
        time.sleep(1)
    start_date = next_date
emociones_df = pd.concat(df_list)

2021-04-01 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-02 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-03 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-04 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-05 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-06 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-07 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-08 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-09 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-10 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-11 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-12 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-13 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-14 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-15 00:00:00
Alegría
Enojo
Miedo
Repulsió

In [11]:
emociones_df.to_csv('../Data/emociones_df.txt', index=False)

In [20]:
emociones_df = pd.read_csv('../Data/emociones_df.txt')

In [6]:
from transformers import BertForMaskedLM, BertTokenizer, BertForSequenceClassification

In [21]:

tokenizer = BertTokenizer.from_pretrained("/home/juanjo/BETO/pytorch/", do_lower_case=False)
model = BertForMaskedLM.from_pretrained("/home/juanjo/BETO/pytorch")
e = model.eval()

Some weights of the model checkpoint at /home/juanjo/BETO/pytorch were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
text = "[CLS] Para solucionar los [MASK] de Chile, el presidente debe [MASK] de inmediato. [SEP]"
masked_indxs = (4,11)

tokens = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
tokens_tensor = torch.tensor([indexed_tokens])

predictions = model(tokens_tensor)[0]

for i,midx in enumerate(masked_indxs):
    idxs = torch.argsort(predictions[0,midx], descending=True)
    predicted_token = tokenizer.convert_ids_to_tokens(idxs[:5])
    print('MASK',i,':',predicted_token)

MASK 0 : ['problemas', 'conflictos', 'asuntos', 'males', 'temas']
MASK 1 : ['renunciar', 'actuar', 'intervenir', 'regresar', 'asumir']
